NaiveBayes CollaborativeFiltering using Laplace Estimator

In [202]:
import operator
import pandas as pd
import numpy as np
import math

In [250]:
#make Arbitrary User-Item Matrix
rating={
    "A":{"블랙머니":5,"카운트다운":4,"나이브스아웃":1},
    "B":{"겨울왕국":2,"카운트다운":1,"나이브스아웃":2},
    "C":{"겨울왕국":1,"쥬만지":4,"나이브스아웃":4,"카운트다운":4},
    "D":{"겨울왕국":3,"쥬만지":2,"블랙머니":3,"나이브스아웃":1},
    "E":{"쥬만지":4,"블랙머니":5,"카운트다운":2}
}
DF=pd.DataFrame(rating).T
DF

,겨울왕국,나이브스아웃,블랙머니,쥬만지,카운트다운
A,NaN,1.0,5.0,NaN,4.0
B,2.0,2.0,NaN,NaN,1.0
C,1.0,4.0,NaN,4.0,4.0
D,3.0,1.0,3.0,2.0,NaN
E,NaN,NaN,5.0,4.0,2.0


In [251]:
#get all of feature
def get_feature(DF,A,I):
    item_rating=dict(DF[DF[I].notna()].get(I))
    item_rating.pop(A,None)
    #print("class=argmax(P(class)",end="")
    #for a,b in dd.items():
    #    print("x(user%s=%d|class)"%(a,b),end="")
    #print(")")
    return item_rating

#get Probability for Useri give rating "j" 
def P_Class(DF,i,j):
    Denominator=0
    nominator=0
    i_rating=dict(DF.T)[i].values
    for rating in i_rating:
        if math.isnan(rating)==False:
            Denominator+=1
            if rating == j:
                nominator+=1
    return nominator/Denominator

#get Probability for Useri give rating "i_rating" when Userj give rating "j_rating"
#using Lapalce Estimator
def conditionality(DF,i,i_rating,j,j_rating):#i=feature j=condition 
    Denominator=5#laplace Estimator X size
    nominator=1#Laplace Estimator +1
    for k,n in enumerate(DF.T[j]):
        if n==j_rating:
            Denominator+=1
            if DF.T[i][k]==i_rating:
                nominator+=1
    return nominator/Denominator

In [252]:
#Preference of User A on Item I
def Predict(DF,A,I):
    df=DF.copy()
    df.at[A,I]=None
    #get class
    features=get_feature(df,A,I)
    #make dictionary "art" to get probabiltity of multi-class rating
    arg={}
    for j_rating in range(1,6):
        prob_C=P_Class(df,A,j_rating)
        for i,i_rating in features.items(): 
            prob_C=prob_C*conditionality(df,i,i_rating,A,j_rating)
        arg[j_rating]=prob_C
    #get argmax from dictionary "arg"
    pre_rating_A_I=max(arg.items(), key=operator.itemgetter(1))[0]
    
    return pre_rating_A_I

In [253]:
#Make new User-Itme Matrix using NaiveBayesCF
items=DF.columns
users=DF.index
new_rating={}
for i in DF.index:
    user_rating={}
    for j in DF.columns:
        user_rating[j]=Predict(DF,i,j)
    new_rating[i]=user_rating

newDF=pd.DataFrame(new_rating).T
print("Conditional Probability Table")
newDF

Conditional Probability Table


,겨울왕국,나이브스아웃,블랙머니,쥬만지,카운트다운
A,1,4,1,1,1
B,1,1,2,2,2
C,4,1,1,4,4
D,1,3,1,1,1
E,5,2,2,2,4
